**Perform a k-fold cross-validation of linear SVC vs CADD**

Configuration
- n_cores: set to the number of CPU cores available on your machine; will spawn this number of parallel processes
- k: number of folds to use in the cross-validation
- confidence: (out of 1; NOT %) for the interval of the mean difference between linear SVC and CADD

In [1]:
n_cores = 32
k = 1000
confidence = 0.999

** No need to edit below here ** Just run each cell

In [2]:
import csv, math, numpy as np, matplotlib.pyplot as plt

from sklearn.svm import SVC
from sklearn.metrics import roc_curve, roc_auc_score, auc, matthews_corrcoef
from sklearn import cross_validation
from multiprocessing import Pool
from scipy import stats

Import the tab-separated values, and partition them according to the class.

0 = non-pathogenic (ESP)

1 = pathogenic (ClinVar)

In [3]:
X = [[], []]

src = open("./all-scores.tsv")
for line in csv.reader(src, dialect="excel-tab"):
    y = int(line[0]);
    map(float, line);
    X[y].append(line[1:]);

for i in range(2):
    X[i] = np.array(X[i], dtype='float')

target = np.array([0]*len(X[0]) + [1]*len(X[1])) # vector of 0s then 1s

Generate k folds with sklearn. This is done for both classes to ensure a balanced input. Training / test groups from each fold are paired (non-pathogenic / pathogenic) to be passed to a parallel pool for cross-validation.

The ROC AUC is calculated using a linear SVC for combination of CADD, RVIS, and PROVEAN. This is compared to the CADD-only AUC for the same test-set.

In [4]:
# select a subset of features to test independent contributions; 0=CADD, 1=RVIS, 2=PROVEAN
featureCols = [0,1,2]

# folds is a (2,1) list of 1 of the folds generated by cross_validation.KFolds on both X[0] and X[1]
# these are paired by class to maintain a balance from each
# foldType corresponds to 0=train and 1=test
def getFoldFeatures(folds, foldType):
    features = X[0][folds[0][foldType]];
    features = np.append(features, X[1][folds[1][foldType]], axis=0)
    return features;

# as with getFoldFeatures, but returns a vector similar to targets, but shorter
def getFoldTargets(folds, foldType):
    return [0]*len(folds[0][foldType]) + [1]*len(folds[1][foldType]);

# function to be mapped over all paired folds
def trainAndValidate(folds):
    X_train = getFoldFeatures(folds, 0);
    y_train = getFoldTargets(folds, 0);
    assert(len((X_train)==len(y_train)))
    
    X_test = getFoldFeatures(folds, 1);
    y_test = getFoldTargets(folds, 1);
    assert(len((X_test)==len(y_test)))

    svc = SVC(kernel='linear').fit(X_train[:,featureCols], y_train);
    decision = svc.decision_function(X_test[:,featureCols])
    
    # X_test[:,0] corresponds to CADD scores alone
    return ({'y': y_test, 'decision': list(decision)}, roc_auc_score(y_test, decision), roc_auc_score(y_test, X_test[:,0]))
    

if __name__ == '__main__':
    p = Pool(n_cores)

    kf = list(); # folds for each class in X; shape=(2,1)
    for i in range(2):
        kf.append(list(cross_validation.KFold(n=len(X[i]), n_folds=k, shuffle=True, random_state=314159))); # constant random_state allows for reproduction of results; 314159 (pi * 10^5) is a nothing-up-my-sleeve number https://en.wikipedia.org/wiki/Nothing_up_my_sleeve_number
    
    pairedFolds = list(); # list of tuples containing fold for each class
    for f in range(k):
        pairedFolds.append((kf[0][f], kf[1][f]));
    
    results = p.map(trainAndValidate, pairedFolds);
    pairedAUC = [];
    for r in results:
        pairedAUC.append([r[1], r[2]])
        
    pairedAUC = np.reshape(np.array(pairedAUC, dtype='float'), (k, 2));
    p.close()
    
    CADD = pairedAUC[:,1]
    CLASSIC = pairedAUC[:,0]
    improvements = CLASSIC - CADD;

Generate statistics of a paired t-test

In [5]:
print('mean AUC CADD:    %0.5f' % CADD.mean())
print('mean AUC CLASSIC: %0.5f' % CLASSIC.mean())
print('n = %u' % len(improvements))
print('mean diff: %0.5f' % (CLASSIC.mean() - CADD.mean()))

t = stats.ttest_rel(CLASSIC, CADD)
df = len(improvements) - 1;
sem = stats.sem(improvements)

print('t-statistic: %0.5f' % t[0])
print('df: %u' % df)
print('p-value: %0.5g (two-sided)' % t[1])

critical = sem * stats.t._ppf((1+confidence)/2, df)
print('%0.1f%% CI: %0.5f %0.5f' % (confidence*100, improvements.mean() - critical, improvements.mean() + critical))

mean AUC CADD:    0.92787
mean AUC CLASSIC: 0.95185
n = 1000
mean diff: 0.02399
t-statistic: 17.05394
df: 999
p-value: 1.9717e-57 (two-sided)
99.9% CI: 0.01934 0.02863


Calculate the Matthews Correlation Coefficient

In [6]:
y_decision = []
decision = []

for r in results:
    y_decision += r[0]['y']
    decision += r[0]['decision']

assert(len(y_decision)==len(decision))
    
y = []
for d in decision:
    y.append(1 if d>0 else 0)
    
print('MCC: %0.2f' % matthews_corrcoef(y_decision, y))

MCC: 0.78


Produce ROC curves

In [7]:
rocCLASSIC = roc_curve(y_decision, decision)
rocCADD = roc_curve(target, list(X[0][:,0]) + list(X[1][:,0]))

plt.figure()

plt.plot(rocCLASSIC[0], rocCLASSIC[1], 'k', label='CLASSIC (%0.1f%%)' % (auc(rocCLASSIC[0], rocCLASSIC[1])*100))
plt.plot(rocCADD[0], rocCADD[1], 'k--', label='CADD (%0.1f%%)' % (auc(rocCADD[0], rocCADD[1])*100))

plt.plot([0, 1], [0, 1], 'k:')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")

plt.show()

Generate a master model from all folds

In [4]:
svc = SVC(kernel='linear').fit(np.append(X[0], X[1], axis=0), target);

In [12]:
print(svc.coef_) # CADD / RVIS / PROVEAN
print(svc.intercept_)

[[ 0.55141463 -0.44124492 -0.2671904 ]]
[-1.94571548]


Output some meta data for inclusion in the manuscript

In [13]:
print('n = %u; %0.2f%% of all CADD-tested sites' % (len(target), len(target)/16334*100))
print('neutral n=%u' % len(X[0]))
print('pathogenic n=%u' % len(X[1]))
print(max(len(X[0]), len(X[1]))/len(target))

import sys, sklearn
print(sys.version)
print(sklearn.__version__)

n = 14435; 88.37% of all CADD-tested sites
neutral n=6968
pathogenic n=7467
0.5172843782473155
3.4.0 (default, Jun 19 2015, 14:20:21) 
[GCC 4.8.2]
0.16.1
